# Calculate Circadian Effects
According to [https://pubmed.ncbi.nlm.nih.gov/8726347/](https://pubmed.ncbi.nlm.nih.gov/8726347/), circadian effects have impact on athletic performance. In order to provide detail to this effect, let us calculate the number of timezones traveled between games for each team.

In [1]:
import requests
import json
import pandas as pd

##### Read in game data

In [2]:
years = ['2015','2016','2017','2018','2019','2020', '2021']
games = {}
for year in years:
    path = '../data/gameData/' + year + '_regSeasonPlays.json'
    print(path)
    with open(path) as f:
        games[year] = json.load(f)

../data/gameData/2015_regSeasonPlays.json
../data/gameData/2016_regSeasonPlays.json
../data/gameData/2017_regSeasonPlays.json
../data/gameData/2018_regSeasonPlays.json
../data/gameData/2019_regSeasonPlays.json
../data/gameData/2020_regSeasonPlays.json
../data/gameData/2021_regSeasonPlays.json


##### Extract data
Get game_id, home and away team, and date for each game

In [3]:
games_list = []

for season in games:
    for g_id in games[season]:
        try:
            games_list.append({
                'game_id': g_id,
                'home_team': games[season][g_id]['home_team'],
                'away_team': games[season][g_id]['away_team'],
                'date': games[season][g_id]['events'][0]['about']['dateTime']
            })
        except:
            pass # game posteponed or canceled (COVID-19 Season)

##### Convert to df, add features

In [4]:
df = pd.DataFrame(games_list)

# convert date string to datetime
df['date'] = pd.to_datetime(df['date'].str[:10]) 

# add location of game column
df['location'] = df['home_team']

# convert df from wide to long
df1 = df[['game_id', 'home_team', 'date', 'location']]
df1.columns = ['game_id', 'team', 'date', 'location']
df2 = df[['game_id', 'away_team', 'date', 'location']]
df2.columns = ['game_id', 'team', 'date', 'location']
df = pd.concat([df1, df2], ignore_index=True)

# merge timezones of team and location
df.sort_values(['team', 'date'], inplace=True)
df['location_prev'] = df.groupby(['team'])['location'].shift(1).fillna(df['location'])
# ...add code

df.head()

,game_id,team,date,location,location_prev
7166,2015020030,Anaheim Ducks,2015-10-11,San Jose Sharks,San Jose Sharks
35,2015020036,Anaheim Ducks,2015-10-12,Anaheim Ducks,San Jose Sharks
48,2015020049,Anaheim Ducks,2015-10-14,Anaheim Ducks,Anaheim Ducks
62,2015020063,Anaheim Ducks,2015-10-16,Anaheim Ducks,Anaheim Ducks
76,2015020077,Anaheim Ducks,2015-10-18,Anaheim Ducks,Anaheim Ducks


pause and retrieve timezone data

In [5]:
# retrieve timezone data from nhl api
r = requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
d = r.json()

# extract data
team_tz = []
for team in d['teams']:
    team_tz.append({
        'team': team['name'],
        'timeZone': team['venue']['timeZone']['tz'],
        'offset': team['venue']['timeZone']['offset']
    })
    
# convert to df
df_tz = pd.DataFrame(team_tz)
df_tz.sort_values('team', inplace=True)

continue with calculating timezone canges

In [6]:
# get timezone/offset of current location
df_tz.columns = ['location', 'timeZone', 'offset']
df = df.sort_values('location').merge(df_tz, how='left', on='location')

# get timezone/offset of current location
df_tz.columns = ['location_prev', 'timeZone_prev', 'offset_prev']
df = df.sort_values('location_prev').merge(df_tz, how='left', on='location_prev')

# calculate change in offset
df = df.sort_values(['team', 'date'])
df['offset_diff'] = df['offset'] - df['offset_prev']


In [7]:
df[df['team'] == 'Carolina Hurricanes'].head(30)

,game_id,team,date,location,location_prev,timeZone,offset,timeZone_prev,offset_prev,offset_diff
7877,2015020009,Carolina Hurricanes,2015-10-08,Nashville Predators,Nashville Predators,CST,-6,CST,-6,0
7656,2015020023,Carolina Hurricanes,2015-10-10,Carolina Hurricanes,Nashville Predators,EST,-5,CST,-6,1
2453,2015020041,Carolina Hurricanes,2015-10-13,Carolina Hurricanes,Carolina Hurricanes,EST,-5,EST,-5,0
2562,2015020060,Carolina Hurricanes,2015-10-16,Detroit Red Wings,Carolina Hurricanes,EST,-5,EST,-5,0
4804,2015020070,Carolina Hurricanes,2015-10-17,Washington Capitals,Detroit Red Wings,EST,-5,EST,-5,0
13524,2015020091,Carolina Hurricanes,2015-10-21,Colorado Avalanche,Washington Capitals,MST,-7,EST,-5,-2
3436,2015020105,Carolina Hurricanes,2015-10-23,Los Angeles Kings,Colorado Avalanche,PST,-8,MST,-7,-1
6274,2015020117,Carolina Hurricanes,2015-10-25,San Jose Sharks,Los Angeles Kings,PST,-8,PST,-8,0
11159,2015020127,Carolina Hurricanes,2015-10-27,Detroit Red Wings,San Jose Sharks,EST,-5,PST,-8,3
4926,2015020137,Carolina Hurricanes,2015-10-29,New York Islanders,Detroit Red Wings,EST,-5,EST,-5,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14274 entries, 11084 to 14273
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   game_id        14274 non-null  object        
 1   team           14274 non-null  object        
 2   date           14274 non-null  datetime64[ns]
 3   location       14274 non-null  object        
 4   location_prev  14274 non-null  object        
 5   timeZone       14274 non-null  object        
 6   offset         14274 non-null  int64         
 7   timeZone_prev  14274 non-null  object        
 8   offset_prev    14274 non-null  int64         
 9   offset_diff    14274 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 1.2+ MB


##### Save to file

In [9]:
result = df.to_json(orient="records")
parsed = json.loads(result)
path = '../data/teamCircadian.json'
with open(path, 'w') as f:
    json.dump(parsed, f, indent=2)

##### To re-open file

In [10]:
with open('../data/teamCircadian.json') as f:
    d = json.load(f)          # read file
df_tmp = pd.DataFrame(d)      # convert to df

In [11]:
df_tmp

,game_id,team,date,location,location_prev,timeZone,offset,timeZone_prev,offset_prev,offset_diff
0,2015020030,Anaheim Ducks,1444521600000,San Jose Sharks,San Jose Sharks,PST,-8,PST,-8,0
1,2015020036,Anaheim Ducks,1444608000000,Anaheim Ducks,San Jose Sharks,PST,-8,PST,-8,0
2,2015020049,Anaheim Ducks,1444780800000,Anaheim Ducks,Anaheim Ducks,PST,-8,PST,-8,0
3,2015020063,Anaheim Ducks,1444953600000,Anaheim Ducks,Anaheim Ducks,PST,-8,PST,-8,0
4,2015020077,Anaheim Ducks,1445126400000,Anaheim Ducks,Anaheim Ducks,PST,-8,PST,-8,0
...,...,...,...,...,...,...,...,...,...,...
14269,2021020126,Winnipeg Jets,1635552000000,San Jose Sharks,Los Angeles Kings,PST,-8,PST,-8,0
14270,2021020139,Winnipeg Jets,1635811200000,Winnipeg Jets,San Jose Sharks,CST,-6,PST,-8,2
14271,2021020157,Winnipeg Jets,1636070400000,Winnipeg Jets,Winnipeg Jets,CST,-6,CST,-6,0
14272,2021020170,Winnipeg Jets,1636156800000,Winnipeg Jets,Winnipeg Jets,CST,-6,CST,-6,0
